# Chunking

This code was taken from Saric (https://github.com/SanjaSaric/HSA-Topics/blob/master/Chunking.ipynb)

In [1]:
from pathlib import Path
import os 
import re

In [2]:
data = 'C:/Users/elisa/DH-MA/data' 

## Laden und sortieren

In [3]:
path_to_corpus = Path(data, '1866-1929') # Ausgangspfad (wird nicht überschrieben)

In [4]:
sorted(os.listdir(path=path_to_corpus))

['church-hatcher-hatcher.txt',
 'fpn-bruce-bruce.txt',
 'fpn-burton-burton.txt',
 'fpn-burtont-burton.txt',
 'fpn-ferebee-ferebee.txt',
 'fpn-hughes-hughes.txt',
 'fpn-lane-lane.txt',
 'fpn-mason-mason.txt',
 'fpn-robinson-robinson.txt',
 'fpn-veney-veney.txt',
 'fpn-washeducation-washing.txt',
 'fpn-washington-washing.txt',
 'nc-jones85-jones85.txt',
 'nc-omarsaid-omarsaid.txt',
 'neh-adams-adams.txt',
 'neh-albert-albert.txt',
 'neh-aleckson-aleckson.txt',
 'neh-alexander-alexander.txt',
 'neh-andersonr-andersonr.txt',
 'neh-andersrob-andersrob.txt',
 'neh-arter-arter.txt',
 'neh-blair-blair.txt',
 'neh-bleby-bleby.txt',
 'neh-bradford-bradford.txt',
 'neh-bragg-bragg.txt',
 'neh-bragg1915-bragg1915.txt',
 'neh-branham-branham.txt',
 'neh-brown80-brown80.txt',
 'neh-brownhal-brownhal.txt',
 'neh-brownsn-brownsn.txt',
 'neh-bruceje-bruceje.txt',
 'neh-bruner-bruner.txt',
 'neh-capehart-capehart.txt',
 'neh-carolinatwin-carolinatwin.txt',
 'neh-charlton-charlton.txt',
 'neh-chesnutt-ch

In [8]:
filenames = [os.path.join(path_to_corpus, fn) for fn in sorted(os.listdir(path_to_corpus))]
filenames

['C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\church-hatcher-hatcher.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\fpn-bruce-bruce.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\fpn-burton-burton.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\fpn-burtont-burton.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\fpn-ferebee-ferebee.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\fpn-hughes-hughes.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\fpn-lane-lane.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\fpn-mason-mason.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\fpn-robinson-robinson.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\fpn-veney-veney.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\fpn-washeducation-washing.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\fpn-washington-washing.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\nc-jones85-jones85.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\1866-1929\\nc-omarsaid-omarsaid.txt',
 'C:\\Users\\elisa\\DH-MA\\da

## Dokumente in chunks teilen

In [9]:
def split_text(filename, n_words):
    """Split a text into chunks approximately `n_words` words in length."""
    input = open(filename, 'r', encoding="utf-8")
    words = " ".join(re.sub(',|\.|\;|\:|\(|\)|\-','',input.read()).split()).split(' ') # remove special charachters and normalize space
    input.close()
    chunks = []
    current_chunk_words = []
    current_chunk_word_count = 0
    for word in words:
        current_chunk_words.append(word)
        current_chunk_word_count += 1
        if current_chunk_word_count == n_words:
            chunks.append(' '.join(current_chunk_words))
            current_chunk_words = []
            current_chunk_word_count = 0
    chunks.append(' '.join(current_chunk_words) )
    return chunks

In [10]:
filenames.sort()

In [11]:
chunk_length = 10000
chunks = []

for filename in filenames:
    chunk_counter = 0
    texts = split_text(filename, chunk_length)
    for text in texts:
        chunk = {'text': text, 'number': chunk_counter, 'filename': filename} # make dictionary with content and information
        chunks.append(chunk)
        chunk_counter += 1
        

Anzahl der Originaldateien:

In [12]:
len(filenames)

121

Anzahl der erzeugten chunks:

In [13]:
len(chunks)

580

Prüfen, ob eine Originaldatei zu kurz war, um sie im Topic Modeling überhaupt zu verwenden. 

In [14]:
i = 0
for chunk in chunks:
    l_chunk = len(chunk['text'].split(' '))
    if l_chunk < 3000 and chunk['number'] == 0:
        i+=1
        print(l_chunk, chunk['filename'],chunk['number'])
print('Anzahl der Dateien, die du entfernen solltest: ', i)

1771 C:\Users\elisa\DH-MA\data\1866-1929\neh-capehart-capehart.txt 0
Anzahl der Dateien, die du entfernen solltest:  1


Hatte eine Datei beispielsweise 110 Tokens, werden 2 chunks produziert: <br>
1) mit 100 Tokens <br>
2) mit 10 Tokens. <br>
Wir möchten diese kurzen chunks ihren vorhergehenden Geschwisterdateien hinzufügen.  

In [15]:
i = 0
for chunk in chunks:
    index = chunks.index(chunk)
    l_chunk = len(chunk['text'].split(' '))
    if l_chunk < 3000 and chunk['number'] != 0:
        i+=1
        chunks[index-1]['text'] = chunks[index-1]['text'] + ' ' + chunk['text']
        print('Chunk ' + chunk['filename'] + str(chunk['number']-1) + ' erweitert mit chunk ' + str(chunk['number']) + ' auf dem Index ' + str(index))
        
print('Anzahl der erweiterten chunks: ' + str(i))

Chunk C:\Users\elisa\DH-MA\data\1866-1929\fpn-lane-lane.txt2 erweitert mit chunk 3 auf dem Index 23
Chunk C:\Users\elisa\DH-MA\data\1866-1929\fpn-mason-mason.txt1 erweitert mit chunk 2 auf dem Index 26
Chunk C:\Users\elisa\DH-MA\data\1866-1929\fpn-veney-veney.txt0 erweitert mit chunk 1 auf dem Index 33
Chunk C:\Users\elisa\DH-MA\data\1866-1929\fpn-washeducation-washing.txt6 erweitert mit chunk 7 auf dem Index 41
Chunk C:\Users\elisa\DH-MA\data\1866-1929\neh-adams-adams.txt0 erweitert mit chunk 1 auf dem Index 56
Chunk C:\Users\elisa\DH-MA\data\1866-1929\neh-albert-albert.txt2 erweitert mit chunk 3 auf dem Index 60
Chunk C:\Users\elisa\DH-MA\data\1866-1929\neh-andersonr-andersonr.txt7 erweitert mit chunk 8 auf dem Index 84
Chunk C:\Users\elisa\DH-MA\data\1866-1929\neh-andersrob-andersrob.txt4 erweitert mit chunk 5 auf dem Index 90
Chunk C:\Users\elisa\DH-MA\data\1866-1929\neh-bleby-bleby.txt3 erweitert mit chunk 4 auf dem Index 100
Chunk C:\Users\elisa\DH-MA\data\1866-1929\neh-branham-b

In [16]:
#chunks

Nun können diejenigen chunks, die bereits zu ihren Geschwisterdateien kopiert wurden, sowie diejenigen chunks, die sehr kurz waren und keine Geschwister hatten (= kurze Originalfiles) gelöscht werden.

In [17]:
i = 0
for chunk in chunks:
    index = chunks.index(chunk)
    l_chunk = len(chunk['text'].split(' '))
    if l_chunk < 3000:
        i+=1
        chunks.remove(chunk)
        
print('Gelöschte chunks: ' + str(i))

Gelöschte chunks: 44


In [18]:
print('Übriggebliebene: ' + str(len(chunks)))

Übriggebliebene: 536


## chunks zu Textdateien speichern

In [19]:
output_dir = 'C:/Users/elisa/DH-MA/data/1866-1929-chunks'

In [20]:
""" Quelle für Code: DARIAH-DE (https://liferay.de.dariah.eu/tatom/index.html)

    for chunk in chunks:
    basename = os.path.basename(chunk['filename'])
    fn = os.path.join(output_dir, "{}{:04d}".format(basename, chunk['number']))
    with open(fn, 'w', encoding='utf-8') as f:
        f.write(chunk['text'])
"""
# umgeändert, sodass valide txt-Dateien als output kommen
for chunk in chunks:
    basename = os.path.basename(chunk['filename'])
    fn_base, fn_ext = os.path.splitext(basename)
    fn = os.path.join(output_dir, "{}_{:04d}{}".format(fn_base, chunk['number'], fn_ext))
    with open(fn, 'w', encoding='utf-8') as f:
        f.write(chunk['text'])

Testen, ob kurze Dateien übriggeblieben sind:

In [21]:
# Test if short files remained
i = 0
for chunkfile in Path(data, output_dir).glob('*.txt'):
    with open(chunkfile, encoding='utf-8') as f:
        text = f.read().split(' ')
        #print(len(text))
        if len(text) < 3000:
            i+=1
            print(chunkfile.name)
print('Übriggebliebene kurze Files: ', i)

Übriggebliebene kurze Files:  0


Wenn ja, dann sollten diese auch manuell entfernt oder zu Geschwisterdateien hinzugefügt werden.
